In [1]:
from sympy import *
from IPython.display import display

h = 5
m = 5
#h, m = symbols("h, m", integer=True, positive=True, nonzero=True)
T4qual = Rational(1, 40)# * Rational(5, 2)
T4prod = Rational(1, 10)# * Rational(5, 2)
q_cv, p_cv, q_rv, p_rv = 4 * T4qual, 1 + 0 * T4prod, 4 *T4qual, 1
d_0v = [1, 0, 0, 0, 0]

q_c, p_c, q_r, p_r = symbols("q_c, p_c, q_r, p_r", real=True, positive=True)
q_cs = Matrix([[*symbols(", ".join("q_{c_"+str(i)+"}" for i in range(1, h+1)), real=True, positive=True)]])
p_cs = Matrix([[*symbols(", ".join("p_{c_"+str(i)+"}" for i in range(1, h+1)), real=True, positive=True)]])
q_rs = Matrix([[*symbols(", ".join("q_{r_"+str(i)+"}" for i in range(1, h+1)), real=True, positive=True)]])
p_rs = Matrix([[*symbols(", ".join("p_{r_"+str(i)+"}" for i in range(1, h+1)), real=True, positive=True)]])

#display(q_cs)

d_0 = Matrix([[*symbols(", ".join("d_0"+str(i) for i in range(1, h+1)), integer=True, positive=True, nonzero=True)]]).T

l = Matrix([[1]*(m-1)+[0]*(h+1-m)])
#L = diag(*l)

i, j, r = symbols("i, j, k", integer=True)

#C_0f = Piecewise((0, i>j),(p_c-p_c*q_c*(Rational(10, 9) - 10**(1+i-h)/9), Eq(i,j)),(p_c*q_c/(10**(j-i-1)), i<j))
#R_0f = Piecewise((0, i>j),(p_r/4-p_r/4*q_r*(Rational(10, 9) - 10**(1+i-h)/9), Eq(i,j)),(p_r*q_r/(4*10**(j-i-1)), i<j))
Cf = Piecewise((0, i>j),(p_c-p_c*q_c*(Rational(10, 9) - 10**(2+i-h)/9), Eq(i,j)),(p_c*q_c/(10**(j-i-1)), i<j))
Rf = Piecewise((0, i>j),(p_r/4-p_r/4*q_r*(Rational(10, 9) - 10**(2+i-h)/9), Eq(i,j)),(p_r*q_r/(4*10**(j-i-1)), i<j))

#Cf = piecewise_fold(Piecewise((C_0f, i<m-1),(0, (i>=m-1) & Ne(i,j)),(1, (i>=m-1) & Eq(i,j))))
#Rf = piecewise_fold(Piecewise((R_0f, i<m-1),(0, (i>=m-1) & Ne(i,j)),(1, (i>=m-1) & Eq(i,j))))

def apply_column_based_effects(L):
    return Matrix([[L[:,iv].subs([(q_c, q_cs[iv]), (p_c, p_cs[iv]), (q_r, q_rs[iv]), (p_r, p_rs[iv])]) for iv in range(L.shape[1])]])

C = FunctionMatrix(h, h, Lambda((j, i), Cf)).as_explicit()
R = FunctionMatrix(h, h, Lambda((j, i), Rf)).as_explicit()
C = apply_column_based_effects(C)
R = apply_column_based_effects(R)

#display(C)
#display(R)

#CR = simplify(C * R)
CR = (C * R)
T = CR * diag(*l) + eye(h) - diag(*l)

#display(T)

def L_eigenvectors(L):
    eigenvectors = []
    for r in range(1, L.shape[0]+1):
        L_minus = L - L[r-1,r-1] * eye(L.shape[0])
        eigenvector = [0] * (r - 1) + [1]
        for jv in range(r+1, L.shape[0]+1):
            s = 0
            for iv in range(1, jv):
                s += L_minus[jv-1,iv-1] * eigenvector[iv-1]
            if L_minus[jv-1,jv-1]==0:
                assert s==0
                eigenvector.append(0)
            else:   
                eigenvector.append(-s/L_minus[jv-1,jv-1])
        eigenvectors.append(eigenvector)
    return Matrix(eigenvectors).T

def L_inverse(L):
    columns = []
    for r in range(1, L.shape[0]+1):
        column = [0] * (r - 1) + [1 / L[r-1,r-1]]
        for jv in range(r+1, L.shape[0]+1):
            s = 0
            for iv in range(1, jv):
                s += L[jv-1,iv-1] * column[iv-1]
            column.append(-s/L[jv-1,jv-1])
        columns.append(column)
    return Matrix(columns).T

def D_inf_val(D):
    return diag(*[1 if D[iv,iv]==1 else 0 for iv in range(D.shape[0])])

def D_inf_sum(D):
    #return simplify(diag(*[0 if D[iv,iv]==1 else 1/(1-D[iv,iv]) for iv in range(D.shape[0])]))
    return diag(*[0 if D[iv,iv]==1 else 1/(1-D[iv,iv]) for iv in range(D.shape[0])])

D = diag(*Matrix.diagonal(T))
#E = simplify(L_eigenvectors(CR))
#E_inv = simplify(L_inverse(E))
E = L_eigenvectors(T)
E_inv = L_inverse(E)

#display(E)
#display(D)
#display(E_inv)

In [2]:
d_inf = E * D_inf_val(D) * E_inv * C * d_0
#display(d_inf)

L = (l * E * D_inf_sum(D) * E_inv * C * d_0)[0,0]

#display(D_inf_sum(D))

val_eq = (Matrix([[1]*h]) - l) * d_inf / (L+(Matrix([[1]*h])*d_0)[0])

In [3]:
#subs_list = [(q_c, q_cv), (p_c, p_cv), (q_r, q_rv), (p_r, p_rv)] + [(k, v) for k, v in zip(d_0, d_0v)]
full_subs_list = [(x, q_cv) for x in q_cs] + [(x, p_cv) for x in p_cs] + [(x, q_rv) for x in q_rs] + [(x, p_rv) for x in p_rs] + [(k, v) for k, v in zip(d_0, d_0v)]

#display(d_inf.subs(subs_list))
#print(float(1/d_inf.subs(subs_list)[m-1]))
#display(Lc.subs(subs_list))
#print(float(Lc.subs(subs_list)))
#display(Lr.subs(subs_list))
#print(float(Lr.subs(subs_list)))

display(d_inf.subs(full_subs_list))
print(float(1/d_inf.subs(full_subs_list)[m-1]))
display(L.subs(full_subs_list))
print(float(L.subs(full_subs_list)))

#derivatives = [q_c, p_c, q_r, p_r]
full_derivatives = [*q_cs] + [*p_cs] + [*q_rs] + [*p_rs]
for x, g in zip(full_derivatives, [val_eq.diff(x) for x in full_derivatives]):
    print("========================================")
    print(x)
    print(float(g.subs(full_subs_list)[0,0]))

Matrix([
[                                     0],
[                                     0],
[                                     0],
[                                     0],
[17399244255868723/18594123149457885623]])

1068.674183545997


74306895620808067600/55782369448373656869

1.3320856814728672
q_{c_1}
0.0025273451513646347
q_{c_2}
0.0007584475412404008
q_{c_3}
0.0009159399492671376
q_{c_4}
0.0011717628405644533
q_{c_5}
0.0
p_{c_1}
0.000214407519184021
p_{c_2}
0.00013146209719392532
p_{c_3}
0.0001656105419780607
p_{c_4}
0.00019072169817397074
p_{c_5}
0.00013097623025580633
q_{r_1}
0.001088712661498811
q_{r_2}
0.0009329958166977052
q_{r_3}
0.0007778553187283808
q_{r_4}
0.000545827686816392
q_{r_5}
0.0
p_{r_1}
0.00015651797710055016
p_{r_2}
0.00018184983167141025
p_{r_3}
0.00016946035956775984
p_{r_4}
0.00015329556062609258
p_{r_5}
0.0


In [4]:
[*q_cs]

[q_{c_1}, q_{c_2}, q_{c_3}, q_{c_4}, q_{c_5}]